In [23]:
import pandas as pd
import numpy as np
import time

In [24]:
train = pd.read_csv("train.csv",index_col = 0)
test = pd.read_csv("test.csv",index_col = 0)
train

,DOM,LP,SP,BT,SqFt,BR,Bth,Gar,GarSp,YrBlt,...,Closing_Date,Age,Area,Zip,Accept_Date,Month_Accept_Date,Week_in_month_Accept_Date,Listing_Date,Month_Listing_Date,Week_in_month_Listing_Date
0,17,769000,759000,CO,1257,2,2,Y,2.0,1999.0,...,2019-01-02,19.0,Santa Teresa,95138,2018-12-03,Dec,1,2018-12-16,Dec,3
1,0,750000,750000,TH,1444,2,2,Y,2.0,1979.0,...,2019-01-02,39.0,Berryessa,95131,2018-12-03,Dec,1,2019-01-02,Jan,1
2,7,750000,785000,TH,1655,3,2,Y,2.0,1982.0,...,2019-01-02,36.0,Santa Teresa,95139,2018-12-03,Dec,1,2018-12-26,Dec,4
3,15,674888,723000,SFH,1408,3,3,NaN,0.0,1953.0,...,2019-01-02,65.0,Alum Rock,95127,2018-12-03,Dec,1,2018-12-18,Dec,3
4,51,649000,636000,CO,922,2,2,NaN,0.0,1988.0,...,2019-01-02,30.0,Campbell,95128,2018-12-03,Dec,1,2018-11-12,Nov,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6611,7,985000,1108000,SFH,1260,3,2,Y,2.0,1955.0,...,2020-12-22,65.0,Santa Clara County,95124,2020-11-22,Nov,4,2020-12-15,Dec,3
6612,4,849000,822000,CO,1622,2,2,Y,2.0,1998.0,...,2020-12-22,22.0,Central San Jose,95112,2020-11-22,Nov,4,2020-12-18,Dec,3
6613,1,715000,766000,CO,1168,2,2,Y,2.0,1996.0,...,2020-12-22,24.0,South San Jose,95136,2020-11-22,Nov,4,2020-12-21,Dec,3
6614,9,887999,894000,SFH,1568,4,2,Y,2.0,1998.0,...,2020-12-22,22.0,South San Jose,95111,2020-11-22,Nov,4,2020-12-13,Dec,2


In [25]:
# drop rows which include San Jose and SJOS - San Jose (train set do not contain)
test = test.drop(test[test['Area'] == 'San Jose'].index)
test = test.drop(test[test['Area'] == 'SJOS - San Jose'].index)

In [26]:
# drop rows in test sets with 'Freq_B', 'Freq_Q', 
test = test.drop(test[test['Freq'] == 'B'].index)
test = test.drop(test[test['Freq'] == 'Q'].index)

In [27]:
# since test set does not have month of accpet date in Oct, add a Oct column in test set equal to 0
test["Month_Accept_Date_Oct"] = [0]*len(test)

In [28]:

y_train = train["SP"]
x_train = pd.get_dummies(train.drop(columns = ['SP','YrBlt','Closing_Date','Accept_Date','Listing_Date']))

y_test = test["SP"]
x_test = pd.get_dummies(test.drop(columns = ['SP','YrBlt','Closing_Date','Accept_Date','Listing_Date']))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
grid_values = {'max_leaf_nodes': [3,6,9],
               'min_samples_leaf': [10],
               'n_estimators': [500,1000,1500,2000,2500],
               'learning_rate': [0.01],
               'random_state': [88]}

tic = time.time()

gbr = GradientBoostingRegressor()
gbr_cv = GridSearchCV(gbr, param_grid=grid_values, scoring = 'neg_root_mean_squared_error', cv=5, n_jobs = -1)
gbr_cv.fit(x_train, y_train)

toc = time.time()
print('time:', round(toc-tic, 2),'s')

In [ ]:
print(gbr_cv.best_params_)

In [ ]:
#calculate OSR2, MSE, MAE
def OSR2(y_train, y_test, y_pred):
    
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(y_train))**2)
                 
    return (1 - SSE/SST)

def MAE(y_test, y_pred):
    
    return (np.mean(abs(y_test - y_pred)))

def RMSE(y_test, y_pred):
    
    return np.sqrt(np.mean((y_test - y_pred)**2))
predict_y = gbr_cv.predict(x_test)
test_y = y_test

print('OSR2', OSR2(y_train,test_y,predict_y))
print('RMSE', RMSE(test_y,predict_y))
print('MAE', MAE(test_y,predict_y))